<a href="https://colab.research.google.com/github/abegpatel/movie-recomendation-system-using-auto-encoder/blob/master/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AUTO ENCODERS:**
.auto encoders
.training of an auto encoders
.overcomplete hidden layers
.sparse autoencoders
.denoising autoencoders
.contractive autoencoders
.stacked autoencoders
.deep autoencoders

**.auto encoders**
used for recomendation system
visibleinput nodes->encoding->hiddenlayer->decoding->visible output layers
.it encoding it self
.self supervised model


.uesd for feature detection
.uesd for powerful recomendation system
.used for encoding
eg..
4 movies as i/p->hidden layer->4 visiable o/p layer
soft max function
->takes highest value
convert highest val-1
else 0



**training of an auto encoders **
1.we start with an array where lines corresponds to user and column
2.the first goes to the network.input vector contains ratings of all movie
3.the i/p vector x is encoded into a vector zof same dimension by mapping function
z=f(wx+b)w->weights
         b-bias
4.z is decoded into the o/p vector y of same dimension as x
5.the reconstruction error d(x,y)=||x-y|| is computedthe goal is to minimize it
6.back_popagated throungh right to left and upadated(gradient descent)
7.repeat 1 to 7
8.read more epochs


**overcomplete hidden layers**
if i/p layer->more hidden layer->o/p layer
it will cheat
and go straight produce o/p
and hidden layer which are left not in use


**.sparse autoencoders**
hidden layer is more than i/p layer
(it will cheat)
.a regularization technique apply (prevent overfitting,stebilizing algorithm)
.it uses ceratin no of nodes at a time


**.denoising autoencoders**
when we have more hidden layer
.a regularization technique
.modified version of i/p value
.randomly assign to 0
.compare o/p to original value
.stochastic auto encoder


**contractive autoencoders**
.a regularization techinque
.they add penalty to loss function


**stacked autoencoders**
.add two hidden layer in auto encoders
.hidden layer->encoding->hidden layer
.directed neural network

**deep autoencoders**
i/p layer->hidden layer1..2..3..->o/p layer(rbm stack)


In [ ]:
!unzip -uq "/content/drive/My Drive/P16-AutoEncoders.zip" -d "/content/drive/My Drive/"
!unzip -uq "/content/drive/My Drive/AutoEncoders/ml-100k.zip" -d "/content/drive/My Drive/AutoEncoders/"
!unzip -uq "/content/drive/My Drive/AutoEncoders/ml-1m.zip" -d "/content/drive/My Drive/AutoEncoders/"

In [ ]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [ ]:
# Importing the dataset
movies = pd.read_csv('/content/drive/My Drive/AutoEncoders/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/content/drive/My Drive/AutoEncoders/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/content/drive/My Drive/AutoEncoders/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')



In [ ]:
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
# Preparing the training set and the test set
training_set = pd.read_csv('/content/drive/My Drive/AutoEncoders/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('/content/drive/My Drive/AutoEncoders/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')


In [ ]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))


In [ ]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)


In [ ]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [ ]:

# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [ ]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss= criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.7430)
epoch: 2 loss: tensor(1.0958)
epoch: 3 loss: tensor(1.0530)
epoch: 4 loss: tensor(1.0383)
epoch: 5 loss: tensor(1.0310)
epoch: 6 loss: tensor(1.0264)
epoch: 7 loss: tensor(1.0237)
epoch: 8 loss: tensor(1.0217)
epoch: 9 loss: tensor(1.0208)
epoch: 10 loss: tensor(1.0194)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0184)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0174)
epoch: 15 loss: tensor(1.0173)
epoch: 16 loss: tensor(1.0167)
epoch: 17 loss: tensor(1.0167)
epoch: 18 loss: tensor(1.0167)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0160)
epoch: 21 loss: tensor(1.0161)
epoch: 22 loss: tensor(1.0158)
epoch: 23 loss: tensor(1.0158)
epoch: 24 loss: tensor(1.0157)
epoch: 25 loss: tensor(1.0156)
epoch: 26 loss: tensor(1.0154)
epoch: 27 loss: tensor(1.0152)
epoch: 28 loss: tensor(1.0149)
epoch: 29 loss: tensor(1.0122)
epoch: 30 loss: tensor(1.0111)
epoch: 31 loss: tensor(1.0101)
epoch: 32 loss: tensor(1.0074)
epoch: 33 loss: t

In [ ]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[(target == 0).unsqueeze(0)] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test loss: tensor(0.9482)
